In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, MaxPooling1D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping
from imblearn.combine import SMOTETomek

In [2]:
data=pd.read_csv('Fraud.csv')
df=data
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6362620 entries, 0 to 6362619
Data columns (total 11 columns):
 #   Column          Dtype  
---  ------          -----  
 0   step            int64  
 1   type            object 
 2   amount          float64
 3   nameOrig        object 
 4   oldbalanceOrg   float64
 5   newbalanceOrig  float64
 6   nameDest        object 
 7   oldbalanceDest  float64
 8   newbalanceDest  float64
 9   isFraud         int64  
 10  isFlaggedFraud  int64  
dtypes: float64(5), int64(3), object(3)
memory usage: 534.0+ MB


In [3]:
print( df.duplicated().any())

False


In [4]:
#checking for class imbalance 
df['isFraud'].value_counts()

isFraud
0    6354407
1       8213
Name: count, dtype: int64

In [5]:
#removing non numerical data
features=df.drop('type',axis=1)
features=features.drop('nameOrig',axis=1)
features=features.drop('nameDest',axis=1)


In [6]:
X=features.drop('isFlaggedFraud',axis=1)
y= df.isFlaggedFraud

In [7]:
not_fraud=features[features['isFraud']==0]
fraud=features[features['isFraud']==1]
#63544
not_fraud_sample=not_fraud.sample(n=8213)
dd=pd.concat([not_fraud_sample,fraud])

In [8]:
X=dd.drop('isFlaggedFraud',axis=1)
y= dd.isFlaggedFraud
#y=np.array(v).reshape(-1,1)#making sure that y is 1D

In [9]:
#training and testing data for validation
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=8)
y_train = y_train.ravel()#flattening

#scaling the data to reduce the impact of outliers

sc = StandardScaler()
X_train_std = sc.fit_transform(X_train)
X_test_std = sc.transform(X_test)

X_train = np.array(X_train).reshape((X_train.shape[0], X_train.shape[1], 1))
X_test = np.array(X_test).reshape((X_test.shape[0], X_test.shape[1], 1))

In [10]:
import keras_tuner as kt
def build_model(hp):
    model = Sequential()
    model.add(LSTM(units=hp.Int('units1', min_value=32, max_value=256, step=32), activation='relu', input_shape=(X_train.shape[1], X_train.shape[2]), return_sequences=True))
    model.add(Dropout(hp.Float('dropout1', 0.2, 0.5, step=0.1)))
    model.add(LSTM(units=hp.Int('units2', min_value=32, max_value=256, step=32), activation='relu', return_sequences=True))
    model.add(Dropout(hp.Float('dropout2', 0.2, 0.5, step=0.1)))
    model.add(LSTM(units=hp.Int('units3', min_value=32, max_value=256, step=32), activation='relu', return_sequences=True))
    model.add(Dropout(hp.Float('dropout3', 0.2, 0.5, step=0.1)))
    model.add(LSTM(units=hp.Int('units4', min_value=32, max_value=256, step=32), activation='relu', return_sequences=True))
    model.add(Dropout(hp.Float('dropout4', 0.2, 0.5, step=0.1)))
    model.add(LSTM(units=hp.Int('units5', min_value=32, max_value=256, step=32), activation='relu'))
    model.add(Dropout(hp.Float('dropout5', 0.2, 0.5, step=0.1)))
    model.add(Dense(1, activation='sigmoid'))
    
    model.compile(optimizer=Adam(learning_rate=hp.Choice('learning_rate', [1e-2, 1e-3, 1e-4])), loss='binary_crossentropy', metrics=['accuracy'])
    return model

In [11]:
#model.compile(optimizer=Adam(learning_rate=0.001), loss='binary_crossentropy', metrics=['accuracy'])
tuner = kt.RandomSearch(build_model, objective='val_accuracy', max_trials=10, executions_per_trial=1, directory='tuner_dir', project_name='fraud_detection')

Reloading Tuner from tuner_dir\fraud_detection\tuner0.json


In [12]:
early_stopping = EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)

# Train the model with early stopping
#history=model.fit(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=1)

# Evaluate the model
#loss, accuracy = model.evaluate(X_test, y_test, verbose=0)
#print(f"Test Accuracy: {accuracy * 100:.2f}%")

In [34]:
tuner.search(X_train, y_train, epochs=10, batch_size=32, validation_split=0.2, callbacks=[early_stopping], verbose=1)

# Get the best model
best_model = tuner.get_best_models(num_models=1)[0]

# Evaluate the best model
loss, accuracy = best_model.evaluate(X_test, y_test, verbose=0)
print(f"Test Accuracy: {accuracy * 100:.2f}%")

Test Accuracy: 99.76%


In [31]:
y_pred = best_model.predict(X_test)
y_pred = (y_pred > 0.5).astype(int)

103/103 [==============================] - 2s 22ms/step


In [41]:
dd['isFraud'].value_counts()

isFraud
0    8213
1    8213
Name: count, dtype: int64

In [37]:
print(confusion_matrix)

[[3278    3]
 [   5    0]]


In [38]:
print(accuracy_score(y_test, y_pred))

0.9975654290931223


In [43]:
best_model.save('fraud_detection_rnn.h5')

C:\Users\shale\AppData\Local\Programs\Python\Python311\Lib\site-packages\keras\src\engine\training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [44]:
Fraud_detect=best_model
#Fraud_detect is model

In [ ]:
#Ans1 I checked for issing values and used standardscaler
#Ans2 I used lstm rnn model

#4 oldbalanceOrg newbalanceOrig
# yes as they  show difference
# company should carefully monitor transactinso and use ai model for prevention.